In [14]:
#conda install -c conda-forge mlxtend

In [15]:
#pip install mlxtend

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [3]:
df=pd.read_csv(r"C:\Users\pc\Downloads\Online Retail.xlsx - Online Retail.csv")

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [5]:
df.shape

(541909, 8)

In [6]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [7]:
# since buyer behaviour differ  from one geography to another and hence we will take one country at a time for this
#some of the description have spaces that need to be removed
df['Description']=df['Description'].str.strip()

In [8]:
#check if an invoice is missing
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [9]:
#drop the rows that dont have invoice number
df.dropna(axis=0,subset=['InvoiceNo'],inplace=True)

In [10]:
#looking at sales for france only for ease
basket=(df[df['Country']=='France']
       .groupby(['InvoiceNo','Description'])['Quantity'].sum()
       .unstack()
       .reset_index().fillna(0)
       .set_index('InvoiceNo'))


In [11]:
basket.shape

(461, 1564)

In [12]:
basket.to_excel(r"D:\sofronics\basket.xlsx")

In [13]:
# making a function which returns 0 or 1
# 0 means item was not in that transaction, 1 means item present in that transaction
def replace_quantity(x):
    if x>=1:
        return 1
    else:
        return 0
#applying encoding
basket_sets=basket.applymap(replace_quantity)

In [14]:
basket_sets.to_excel(r"D:\sofronics\basket_data.xlsx")

In [15]:
#delete POSTAGE item from the data.it is included in many bills to add delivery cahrge
basket_sets.drop('POSTAGE',inplace=True,axis=1)

In [16]:
#generate frequent item sets that have a support of atleast 7%
#(this number was chosen so that i could get enough useful examples)
frequent_itemsets=apriori(basket_sets,min_support=0.07,use_colnames=True)

C:\Users\pc\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [17]:
#the final step is to generate the rules with their corresponding support,confidence and lift
rules=association_rules(frequent_itemsets,metric='lift',min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.143167,0.117137,0.075922,0.530303,4.527217,0.059152,1.879645,0.909295
1,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.117137,0.143167,0.075922,0.648148,4.527217,0.059152,2.435209,0.882485
2,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.145336,0.143167,0.086768,0.597015,4.170059,0.065961,2.126215,0.889467
3,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.143167,0.145336,0.086768,0.606061,4.170059,0.065961,2.169531,0.887215
4,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.145336,0.117137,0.088937,0.611940,5.224157,0.071913,2.275071,0.946081
5,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.117137,0.145336,0.088937,0.759259,5.224157,0.071913,3.550142,0.915863
6,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER CUPS),0.112798,0.117137,0.086768,0.769231,6.566952,0.073555,3.825741,0.955501
7,(SET/6 RED SPOTTY PAPER CUPS),(SET/20 RED RETROSPOT PAPER NAPKINS),0.117137,0.112798,0.086768,0.740741,6.566952,0.073555,3.422064,0.960197
8,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER PLATES),0.112798,0.108460,0.086768,0.769231,7.092308,0.074534,3.863341,0.968215
9,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.108460,0.112798,0.086768,0.800000,7.092308,0.074534,4.436009,0.963504


In [18]:
#lift" is a measure of how much more likely item B is to be purchased when item A is purchased, 
#compared to when item B is purchased without item A.

In [19]:
#In simpler terms:

#If lift = 1, it means there is no association between items A and B.
#If lift > 1, it means that items A and B are more likely to be bought together.
#The higher the lift value, the stronger the association.
#If lift < 1, it means that items A and B are less likely to be bought together. 
#This indicates a negative correlation between the items.

In [20]:
#wecan filter the data frame using standard pandas code
#in this case, look for a large lift(6) and high confidence(.8):
rules[(rules['lift']>=6) & (rules['confidence']>=0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.108460,0.117137,0.104121,0.960000,8.195556,0.091417,22.071584,0.984793
11,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.117137,0.108460,0.104121,0.888889,8.195556,0.091417,8.023861,0.994472
12,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.086768,0.117137,0.084599,0.975000,8.323611,0.074435,35.314534,0.963457
13,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.104121,0.112798,0.084599,0.812500,7.203125,0.072854,4.731743,0.961259
14,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.086768,0.108460,0.084599,0.975000,8.989500,0.075188,35.661605,0.973202


In [21]:
#export association rules to excel
rules.to_excel(r"D:\sofronics\output_france.xlsx")